In [5]:
import json
import os
import glob
import pandas as pd
import getpass
import forecastio
import xml.etree.ElementTree
from bs4 import BeautifulSoup
import xmltodict
from googlemaps import googlemaps
import forecastio
import matplotlib.pyplot as plt
%matplotlib inline
import pytz


In [2]:
failed_ls = ['002_2014','489_2015',
            '100,2015','005_2015','026_2015','002_2015','494_2016','004_2014','470_2016','018_2016',
            '019_2015','005_2016','002_2016','478_2016','004_2015','489_2016','500_2015',
            '014_2015','464_2016','493_2016','478_2015']

In [3]:
tournaments = glob.glob('*')
tournaments

['005_2016_tournsum.json',
 '005_2016_setup.xml',
 '005_2016_setup.json',
 '005_2016_money.json',
 '005_2016_leaderboard-v2.json',
 '005_2016_player_stats.json',
 '005_2016_coursestat.json',
 '005_2016_teetimes.json',
 '493_2016_playoff_info.json',
 '005_2016_course.json',
 'PlayerScoreCards',
 '005_2016_playoff_info.json']

In [36]:
prefix = '464_2016'
os.chdir('/home/valesco/Datasets/PGA_Data/PGA/' + prefix + '/')

data_ls = ['course','teetimes','leaderboard-v2','player_stats','setup','coursestat','setup.xml','tournsum']

for e, data in enumerate(data_ls):
    if '.xml' in data:
        temp = prefix + '_' + data
    else:
        temp = prefix + '_' + data + '.json'
    data_ls[e] = temp
    

course_json = open(data_ls[0])
course_data = json.load(course_json)
tee_json = open(data_ls[1])
tee_data = json.load(tee_json)
leaderboard_json = open(data_ls[2])
leaderboard_data = json.load(leaderboard_json)
stats_json = open(data_ls[3])
stats_data = json.load(stats_json)
setup_json = open(data_ls[4])
setup2_data = json.load(setup_json)
#course_stats_json = open(data_ls[5])
#course_stats_data = json.load(course_stats_json)
with open(data_ls[6]) as fd:
    setup_data = xmltodict.parse(fd.read())
tournsum_json = open(data_ls[7])
tournsum_data = json.load(tournsum_json)
course_loc_json = open('/home/valesco/Datasets/PGA_Data/courselocations.json')
course_loc_data = json.load(course_loc_json)

In [44]:
tee_data['tournament']['rounds'][0]['courses'][0]['Front']

'36'

In [418]:
setup2_data['trn']['rnds'][0]['courses'][0]['teetimes']



[{'groups': [{'id': '1',
    'players': ['25562', '23638', '28486'],
    'start': {'dateDD': '16',
     'dateMM': '10',
     'dateYYYY': '2014',
     'timeHH': '06',
     'timeMM': '50',
     'timeSS': '00',
     'utcOffset': '-07'}},
   {'id': '2',
    'players': ['23353', '24846', '31193'],
    'start': {'dateDD': '16',
     'dateMM': '10',
     'dateYYYY': '2014',
     'timeHH': '07',
     'timeMM': '00',
     'timeSS': '00',
     'utcOffset': '-07'}},
   {'id': '3',
    'players': ['24361', '22378', '30944'],
    'start': {'dateDD': '16',
     'dateMM': '10',
     'dateYYYY': '2014',
     'timeHH': '07',
     'timeMM': '10',
     'timeSS': '00',
     'utcOffset': '-07'}},
   {'id': '4',
    'players': ['23135', '26679', '21961'],
    'start': {'dateDD': '16',
     'dateMM': '10',
     'dateYYYY': '2014',
     'timeHH': '07',
     'timeMM': '20',
     'timeSS': '00',
     'utcOffset': '-07'}},
   {'id': '5',
    'players': ['34099', '20229', '20157'],
    'start': {'dateDD': '16',
 

In [321]:
hole_keys = tee_023_data['tournament']['rounds'][0]['courses'][0]['holes'][1].keys()

coord_dict = {}
for key in hole_keys:
    if isinstance(tee_023_data['tournament']['rounds'][0]['courses'][0]['holes'][1][key], dict):
        if 'x' in tee_023_data['tournament']['rounds'][0]['courses'][0]['holes'][1][key].keys():
            x = tee_023_data['tournament']['rounds'][0]['courses'][0]['holes'][1][key]['x']
            y = tee_023_data['tournament']['rounds'][0]['courses'][0]['holes'][1][key]['y']
            z = tee_023_data['tournament']['rounds'][0]['courses'][0]['holes'][1][key]['z']
            coord_dict[key] = [x,y,z]
    elif isinstance(tee_023_data['tournament']['rounds'][0]['courses'][0]['holes'][1][key], list):
        try:
            if 'x' in tee_023_data['tournament']['rounds'][0]['courses'][0]['holes'][1][key][0].keys():
                for i in range(len(tee_023_data['tournament']['rounds'][0]['courses'][1]['holes'][1][key])):
                    x = tee_023_data['tournament']['rounds'][0]['courses'][0]['holes'][1][key][i]['x']
                    y = tee_023_data['tournament']['rounds'][0]['courses'][0]['holes'][1][key][i]['y']
                    z = tee_023_data['tournament']['rounds'][0]['courses'][0]['holes'][1][key][i]['z'] 
                    coord_dict[key + str(i)] = [x,y,z]
        except:
            pass
    #else:
        #coord_dict[key] = tee_023_data['tournament']['rounds'][0]['courses'][0]['holes'][0][key]
    
coord_dict

{'BoxFrontOfGreen': ['0', '0', '0'],
 'ClosestEdge': ['12365.7', '11871.1', '853.526'],
 'FrontOfGreen': ['12350', '-1', '871.394'],
 'LowerLeftBox': ['0', '0', '0'],
 'PinLocation': ['12351', '11871.4', '853.773'],
 'Tee': ['11513.4', '10834.4', '899.728'],
 'TeeLocation': ['11513.4', '10834.4', '899.728'],
 'UpperRightBox': ['0', '0', '0']}

In [320]:
coord_dict['FrontOfGreen'][1] = '0'
coord_dict.pop('FrontOfGreen')


for k in coord_dict:
    try:
        x = float(coord_dict[k][0])
        y = float(coord_dict[k][1])
    except:
        pass
    plt.scatter(x,y)
    plt.annotate(k, xy = (x,y))
    
plt.show()

In [297]:
tee_023_data['tournament']['rounds'][0]['courses'][0]['holes'][0]['Fairways'][0]['x']


'10811.8'

In [323]:
coord_dict

{'BoxFrontOfGreen': ['0', '0', '0'],
 'ClosestEdge': ['12365.7', '11871.1', '853.526'],
 'FrontOfGreen': ['12350', '-1', '871.394'],
 'LowerLeftBox': ['0', '0', '0'],
 'PinLocation': ['12351', '11871.4', '853.773'],
 'Tee': ['11513.4', '10834.4', '899.728'],
 'TeeLocation': ['11513.4', '10834.4', '899.728'],
 'UpperRightBox': ['0', '0', '0']}

In [322]:
tee_023_data['tournament']['rounds'][0]['courses'][0]['holes'][1]

{'BoxFrontOfGreen': {'x': '0', 'y': '0', 'z': '0'},
 'BoxRotationAngle': '0',
 'ClosestEdge': {'x': '12365.7', 'y': '11871.1', 'z': '853.526'},
 'Fairways': [{'seq': '1', 'x': '12124.7', 'y': '11515.3', 'z': '858.793'}],
 'FrontOfGreen': {'x': '12350', 'y': '-1', 'z': '871.394'},
 'HYardage': [],
 'HoleNumber': '2',
 'IsCoordinateValid': 'Yes',
 'LowerLeftBox': {'x': '0', 'y': '0', 'z': '0'},
 'Par': '4',
 'PinLocation': {'x': '12351', 'y': '11871.4', 'z': '853.773'},
 'TYardage': [{'desc': 'CARTPATH CORNER',
   'dist': '12114',
   'num': '1',
   'x': '12225',
   'y': '11549',
   'z': '854'},
  {'desc': '250', 'dist': '9000', 'num': '250'},
  {'desc': '275', 'dist': '9900', 'num': '275'},
  {'desc': '300', 'dist': '10800', 'num': '300'},
  {'desc': '325', 'dist': '11700', 'num': '325'},
  {'desc': '350', 'dist': '12600', 'num': '350'}],
 'Tee': {'CalculatedYardage': '445',
  'DistanceToPin': '455',
  'TType': 'YCH',
  'x': '11513.4',
  'y': '10834.4',
  'z': '899.728'},
 'TeeLocation':